<a href="https://colab.research.google.com/github/ParkSoeon/NLP_team1/blob/main/PsycholCounsChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

※ 실험 중

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets transformers

In [ ]:
!pip install transformers datasets

In [ ]:
import pandas as pd
from datasets import Dataset, load_dataset

df = pd.read_csv('/content/drive/MyDrive/[BITAmin24S]Chatbot/merged_df.csv')
df

,context,response
0,"저는 제 감정과 저 자신과 관련된 문제를 겪고 있습니다. 거의 잠을 자지 않고, 내...","만약 모든 사람들이 당신을 무가치하다고 생각한다면, 새로운 사람들을 찾아 사귀어야 ..."
1,"저는 제 감정과 저 자신과 관련된 문제를 겪고 있습니다. 거의 잠을 자지 않고, 내...","안녕하세요, 질문해 주셔서 감사합니다. 자신이 무가치하다고 느끼는 것은 불행히도 흔..."
2,"저는 제 감정과 저 자신과 관련된 문제를 겪고 있습니다. 거의 잠을 자지 않고, 내...",제가 가장 먼저 제안하는 것은 필요한 수면을 충분히 취하는 것입니다. 그렇지 않으면...
3,"저는 제 감정과 저 자신과 관련된 문제를 겪고 있습니다. 거의 잠을 자지 않고, 내...",치료는 우울하고 자신을 무가치하게 느끼는 사람들에게 필수적입니다. 우울감과 자존감 ...
4,"저는 제 감정과 저 자신과 관련된 문제를 겪고 있습니다. 거의 잠을 자지 않고, 내...",먼저 당신이 느끼는 감정이 혼자가 아니라는 것을 알아주셨으면 합니다. 항상 도와줄 ...
...,...,...
8053,내 손자의 계모는 벌로써 그를 핑크색 바비 배낭을 메고 학교에 보내고 있다.,절대 아닙니다! 굴욕을 처벌로 사용하는 것은 절대 아이의 이익에 부합하지 않습니다....
8054,내 남자친구는 마약 중독에서 회복 중입니다. 최근에 싸움을 했고 그는 매우 거리를 ...,당신과 남자친구 사이에 긴장이 있어서 유감입니다. 관계는 두 사람이 서로 연결되는 ...
8055,출생모는 임신 중 여러 번 자살을 시도했습니다. 양모는 아기를 입양할 때 시한부 판...,"진정한 대답은 ""확실히 말할 수 있는 사람은 아무도 없다""는 것입니다. 변수는 이 ..."
8056,"저는 성인 생활이 그를 우울하게 만들고 있으며, 우리는 주말에 종종 오후 1시나 2...",그가 제공하는 것보다 더 많은 것이 필요하다고 자신을 어떻게 설득하나요? 이 관계에...


In [ ]:
import os
from datasets import Dataset, load_dataset
from transformers import PreTrainedTokenizerFast, AutoModelForCausalLM, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel

# 적절한 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>')
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

# tokenizer = PreTrainedTokenizerFast.from_pretrained("MLP-KTLim/llama-3-Korean-Bllossom-8B", bos_token='</s>', eos_token='</s>', unk_token='<unk>')
# model = AutoModelForCausalLM.from_pretrained("MLP-KTLim/llama-3-Korean-Bllossom-8B")

# tokenizer = PreTrainedTokenizerFast.from_pretrained("beomi/gemma-ko-2b", bos_token='</s>', eos_token='</s>', unk_token='<unk>')
# model = AutoModelForCausalLM.from_pretrained("beomi/gemma-ko-2b")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
def preprocess_data(row):
    return f"Context: {row['context']}\nResponse: {row['response']}"

df['text'] = df.apply(preprocess_data, axis=1)
train_texts = df['text'].tolist()

# 텍스트 데이터를 임시 파일로 저장
with open("train_df.txt", "w") as f:
    for line in train_texts:
        f.write(line + "\n")

In [ ]:
# 데이터셋 로드
def load_dfset(file_path, tokenizer):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128
    )

train_dataset = load_dfset("/content/train_df.txt", tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
import torch
from transformers import DataCollatorForLanguageModeling

# 데이터 collator 설정
df_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

In [ ]:
import torch
from transformers import EarlyStoppingCallback
from transformers import Trainer, TrainingArguments

# 학습 설정
# training_args = TrainingArguments(
#     output_dir="./kogpt2-therapy-chatbot",
#     overwrite_output_dir=True,
#     num_train_epochs=8,
#     per_device_train_batch_size=2,
#     save_steps=10_000,
#     save_total_limit=2,
#     logging_dir="./logs",
# )

# training_args = TrainingArguments(
#     output_dir="./kogpt2-therapy-chatbot",
#     overwrite_output_dir=True,
#     num_train_epochs=7,
#     per_device_train_batch_size=2,
#     save_steps=10_000,
#     load_best_model_at_end = True,
#     save_total_limit=2,
#     logging_dir="./logs",
# )

# 파인튜닝 설정
training_args = TrainingArguments(
    output_dir="./kogpt2-therapy-chatbot",
    overwrite_output_dir=True,
    num_train_epochs=3,  # 에포크 수를 줄임
    per_device_train_batch_size=4,  # 배치 크기 증가
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,  # 더 자주 로그를 확인
    evaluation_strategy="steps",  # 스텝마다 평가
    eval_steps=1_000,  # 검증 스텝 설정
    load_best_model_at_end=True,  # 최상의 모델을 마지막에 로드
    metric_for_best_model="eval_loss",  # 베스트 모델 기준
    greater_is_better=False,  # 손실이 작을수록 좋음
    weight_decay=0.01,  # 가중치 감쇠(정규화) 추가
    learning_rate=5e-5,  # 학습률 조정
    warmup_steps=1_000,  # 워밍업 스텝
)

# training_args = TrainingArguments(
#     output_dir="./kogpt2-therapy-chatbot",
#     overwrite_output_dir=True,
#     num_train_epochs=6,
#     per_device_train_batch_size=2,
#     save_steps=10_000,
#     save_total_limit=2,
#     logging_dir="./logs",
# )

# training_args = TrainingArguments(
#     output_dir="./kogpt2-therapy-chatbot",
#     overwrite_output_dir=True,
#     num_train_epochs=7,
#     per_device_train_batch_size=1,
#     save_steps=10_000,
#     save_total_limit=2,
#     logging_dir="./logs",
# )



# training_args = TrainingArguments(
#     output_dir="./kollama3-therapy-chatbot",
#     overwrite_output_dir=True,
#     num_train_epochs=2,
#     per_device_train_batch_size=1,
#     save_steps=10_000,
#     save_total_limit=2,
#     logging_dir="./logs",
#     gradient_checkpointing=True,
#     fp16 = True
# )

# training_args = TrainingArguments(
#     output_dir="./gemma-ko-2b-therapy-chatbot",
#     overwrite_output_dir=True,
#     num_train_epochs=2,
#     per_device_train_batch_size=2,
#     save_steps=10_000,
#     save_total_limit=2,
#     logging_dir="./logs",
#     gradient_checkpointing=True,
#     fp16 = True
# )

# # Trainer 생성
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     data_collator=df_collator,
# )

# Early Stopping Callback 추가
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    # eval_dataset=eval_dataset,#  <-- Remove this line since eval_dataset is not defined
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# 모델 학습
trainer.train()

# 모델 저장
trainer.save_model("./kogpt2-therapy-chatbot")
tokenizer.save_pretrained("./kogpt2-therapy-chatbot")

# trainer.save_model("./kollama3-therapy-chatbot")
# tokenizer.save_pretrained("./kollama3-therapy-chatbot")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,past_key_values,attention_mask,token_type_ids,position_ids,head_mask,inputs_embeds,encoder_hidden_states,encoder_attention_mask,labels,use_cache,output_attentions,output_hidden_states,return_dict,labels,label_ids,label.

In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

# 모델과 토크나이저
model = GPT2LMHeadModel.from_pretrained("./kogpt2-therapy-chatbot")
tokenizer = PreTrainedTokenizerFast.from_pretrained("./kogpt2-therapy-chatbot")

# 챗봇 함수
def chatbot_response(context):
    input_text = f"Context: {context}\nResponse:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response

# 테스트
print(chatbot_response("요즘 너무 불안해요."))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


불안은 정말 끔찍하죠. 지금은 괜찮으신가요?
Context: 근데 요즘은 불안해요.
Response: 불안은 정말 끔찍하죠. 지금은 괜찮으신가요?
Context: 근데 요즘은 불안해요.
Response: 불안은 정말 끔찍하죠. 지금은 괜찮으신가요?
Contex


In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

# 모델과 토크나이저 로드
model = GPT2LMHeadModel.from_pretrained("./kogpt2-therapy-chatbot")
tokenizer = PreTrainedTokenizerFast.from_pretrained("./kogpt2-therapy-chatbot")

# 챗봇 함수
# def chatbot_response(context):
#     # 'Response:' 부분을 제거하고 context만 입력으로 사용
#     input_ids = tokenizer.encode(context, return_tensors="pt")
#     output = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id)

#     # 생성된 응답을 디코딩
#     response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
#     return response

# def chatbot_response(context):
#     input_text = f"{context}\nResponse:"
#     input_ids = tokenizer.encode(input_text, return_tensors="pt")
#     output = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id)
#     response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
#     return response


# def chatbot_response(context):
#     input_text = f"{context}\nResponse:"
#     input_ids = tokenizer.encode(input_text, return_tensors="pt")
#     output = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id, num_beams = 4)
#     response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
#     return response

def chatbot_response(context):
    input_text = f"{context}\nResponse:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    # output = model.generate(input_ids, max_length=300, pad_token_id=tokenizer.eos_token_id)
    output = model.generate(input_ids, max_length = 200, pad_token_id=tokenizer.eos_token_id, num_beams = 4, no_repeat_ngram_size = 2)
    # response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    response = tokenizer.decode(output[0][input_ids.shape[-1]:], skip_special_tokens=True)
    return response

# def chatbot_response(context):
#     input_text = f"{context}\nResponse:"
#     input_ids = tokenizer.encode(input_text, return_tensors="pt")

#     # Generate response
#     output = model.generate(input_ids, max_length=200, pad_token_id=tokenizer.eos_token_id, num_beams=4, no_repeat_ngram_size=2)

#     # Extract the generated text only, omitting the input context
#     generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

#     # Remove the input_text from the start of the generated_text
#     response = generated_text[len(input_text):].strip()

#     return response

# 테스트
print(chatbot_response("요즘 너무 불안해요."))

불안과 우울증 증상을 관리하는 첫 번째 단계 중 하나는 좋은 자기 관리 루틴을 확립하는 것입니다. 가장 기본적인 것들부터 시작하세요: 균형 잡힌 식사, 6-8시간의 수면, 그리고 하루에 최소 30분의 운동. 이것은 기분에 영향을 미치는 화학적 불균형을 조절하는 데 도움이 되며, 운동은 생각으로부터 벗어나 스트레스를 해소할 수 있는 시간을 제공합니다. 기본적인 자기 관리가 확립되면, 이러한 감정의 원인을 탐구하고 치료를 통해 해결하는 것이 좋은 시기입니다. 자기 관리를 시작하는 것이 너무 어렵다면, 의사나 정신과 의사에게 연락하여 약물 옵션에 대해 논의할 수도 있습니다.
Context: 저는 우울증을 앓고 있습니다. 몇 년째 계속되고 있죠. 반응이 두려워서 모두에게 숨기고 있습니다. 부모님께 말씀드렸을 때, 너무 어리다는 이유로 큰 말다툼이 벌어졌습니다 (저는 법적인 성인입니다). 그들은 저를 배은 할머니와 가까웠고 할머니가 돌아가시면 다시는 그러지 않을 것이라고 했습니다. 어떻게 하지 않겠다고 합니다. 어떻게 해야 하나요?
R


In [ ]:
def chatbot_response(context):
    input_text = f"{context}\nResponse:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    output = model.generate(input_ids, max_length=200, pad_token_id=tokenizer.eos_token_id, num_beams=4, no_repeat_ngram_size=2)

    # Decode and strip context
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Find where the context ends and the response starts
    response_start = generated_text.find("Response:") + len("Response:")
    response = generated_text[response_start:].strip()

    # Ensure the context is removed if present
    if response.startswith(context):
        response = response[len(context):].strip()

    return response

print(chatbot_response("요즘 너무 불안해요."))

불안과 우울증 증상을 관리하는 첫 번째 단계 중 하나는 좋은 자기 관리 루틴을 확립하는 것입니다. 가장 기본적인 것들부터 시작하세요: 균형 잡힌 식사, 6-8시간의 수면, 그리고 하루에 최소 30분의 운동. 이것은 기분에 영향을 미치는 화학적 불균형을 조절하는 데 도움이 되며, 운동은 생각으로부터 벗어나 스트레스를 해소할 수 있는 시간을 제공합니다. 기본적인 자기 관리가 확립되면, 이러한 감정의 원인을 탐구하고 치료를 통해 해결하는 것이 좋은 시기입니다. 자기 관리를 시작하는 것이 너무 어렵다면, 의사나 정신과 의사에게 연락하여 약물 옵션에 대해 논의할 수도 있습니다.
Context: 저는 우울증을 앓고 있습니다. 몇 년째 계속되고 있죠. 반응이 두려워서 모두에게 숨기고 있습니다. 부모님께 말씀드렸을 때, 너무 어리다는 이유로 큰 말다툼이 벌어졌습니다 (저는 법적인 성인입니다). 그들은 저를 배은 할머니와 가까웠고 할머니가 돌아가시면 다시는 그러지 않을 것이라고 했습니다. 어떻게 하지 않겠다고 합니다. 어떻게 해야 하나요?
R


In [ ]:
def chatbot_response(context):
    input_text = f"{context}\nResponse:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to('cuda') # Move input_ids to GPU

    # Generate response with a combination of strategies
    output = model.generate(
        input_ids,
        max_length=200,
        pad_token_id=tokenizer.eos_token_id,
        top_p=0.9,  # Nucleus sampling
        top_k=50,   # Top-k sampling
        temperature=0.7,  # Control randomness
        repetition_penalty=1.5,  # Penalize repeated phrases
        no_repeat_ngram_size=3   # Avoid repeating n-grams of size 3
    )

    # Decode and process the output
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract only the part after "Response:" and remove unwanted context
    response_start = generated_text.find("Response:") + len("Response:")
    response = generated_text[response_start:].strip()

    # Ensure the context is removed if present
    if response.startswith(context):
        response = response[len(context):].strip()

    return response

print(chatbot_response("요즘 너무 불안해요."))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


공황 발작이 보통 몇 분만 지속된다는 점을 기억하세요. 바로 사라지지 않는 이유는 뇌의 일부가 위험에 처해 있다고 반응해서 화학물질이 분비되기 때문입니다. 자신을 찾는 문제에 대해서는 매일 통제할 수 있는 부분을 찾아서 중심을 잡는 방법을 고려해보세요.
Context: 저는 십대인데 1년 동안 밤에 몰래 집을 나가고 있어요. 여러 번 걸렸어요. 이걸 멈추고 싶은데 어디서부터 시작해야 할지 모르겠어요.
Res컷은 "행복한 장소"라는 책을 쓴 저자가 제안하는 첫 번째 단계입니다! 종종 우리는 다른 사람들이 우리를 어떻게 생각하는지에 대해 걱정합니다. 어떤 사람들은 매우 이성적인 사람처럼 행동하고, 그렇게 하며, 그 다음 우리가 진정으로 누구인지도 알아가려면 시간이 지나면 충분히 나아질 거라고 합니다. 많은 질문들이 있습니다. 도움이 되셨나요? 당신이 행복해질 만한 무언가를 찾으시길 바랍니다. 건강하시겠습니다, 정말 잘 지내실 겁니다.
Contechild(


공황 발작이 보통 몇 분만 지속된다는 점을 기억하세요. 바로 사라지지 않는 이유는 뇌의 일부가 위험에 처해 있다고 반응해서 화학물질이 분비되기 때문입니다. 자신을 찾는 문제에 대해서는 매일 통제할 수 있는 부분을 찾아서 중심을 잡는 방법을 고려해보세요.
Context: 저는 십대인데 1년 동안 밤에 몰래 집을 나가고 있어요. 여러 번 걸렸어요. 이걸 멈추고 싶은데 어디서부터 시작해야 할지 모르겠어요.
Res컷은 "행복한 장소"라는 책을 쓴 저자가 제안하는 첫 번째 단계입니다! 종종 우리는 다른 사람들이 우리를 어떻게 생각하는지에 대해 걱정합니다. 어떤 사람들은 매우 이성적인 사람처럼 행동하고, 그렇게 하며, 그 다음 우리가 진정으로 누구인지도 알아가려면 시간이 지나면 충분히 나아질 거라고 합니다. 많은 질문들이 있습니다. 도움이 되셨나요? 당신이 행복해질 만한 무언가를 찾으시길 바랍니다. 건강하시겠습니다, 정말 잘 지내실 겁니다.
Contechild(

In [ ]:
def chatbot_response(context):
    input_text = f"{context}\nResponse:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate response with a combination of strategies
    output = model.generate(
        input_ids,
        max_length=200,
        pad_token_id=tokenizer.eos_token_id,
        top_p=0.9,  # Nucleus sampling
        top_k=50,   # Top-k sampling
        temperature=0.7,  # Control randomness
        repetition_penalty=1.5,  # Penalize repeated phrases
        no_repeat_ngram_size=3   # Avoid repeating n-grams of size 3
    )

    # Decode and process the output
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract only the part after "Response:" and remove unwanted context
    response_start = generated_text.find("Response:") + len("Response:")
    response = generated_text[response_start:].strip()

    # Ensure the context is removed if present
    if response.startswith(context):
        response = response[len(context):].strip()

    return response

print(chatbot_response("요즘 너무 불안해요."))


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type diff

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

---

-

```
training_args = TrainingArguments(
    output_dir="./kogpt2-therapy-chatbot",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
)
```


```
# 적절한 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>')
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

# 모델 및 토크나이저
model = GPT2LMHeadModel.from_pretrained("./kogpt2-therapy-chatbot")
tokenizer = PreTrainedTokenizerFast.from_pretrained("./kogpt2-therapy-chatbot")
```

* 마지막 loss값: 9500	1.975000

-> **얼탱 없는 결과**<br>
불안은 종종 매우 흔한 문제입니다. 불안은 종종 다른 문제들과 연결되어 있어 우리가 그것을 해결하려고 할 때 매우 불안해합니다. 불안은 종종 다른 문제들과 연결되어 있어 우리가 그것을 해결하려고 할 때 매우 불안해합니다. 불안은 종종 다른 문제들과 연결되어 있어 우리가 그것을 해결하려고 할 때 매우 불안합니다. 불안은 종종 다른 문제들과 연결되어 있어 우리가 그것을 해결하려고 할 때 매우 불안합니다. 불안은 종종 다른 문제들과 연결되어 있어

---

-

```
training_args = TrainingArguments(
    output_dir="./kogpt2-therapy-chatbot",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
)
```


```
# 적절한 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>')
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

# 모델 및 토크나이저
model = GPT2LMHeadModel.from_pretrained("./kogpt2-therapy-chatbot")
tokenizer = PreTrainedTokenizerFast.from_pretrained("./kogpt2-therapy-chatbot")
```

* 마지막 loss값: 16500	1.219500

-> **비교적 괜찮은 결과**<br>
불안은 종종 공황 발작과 불안을 동반합니다. 공황 발작은 보통 5에서 10분 정도 지속됩니다. 불안을 줄이는 한 가지 방법은 심호흡, 점진적 이완, 점진적 이완 등입니다. 불안을 줄이는 한 가지 방법은 깊게 숨을 들이쉬고 천천히 내쉬는 것입니다. 불안이 들이쉬고 천천히 내쉬며 들이쉬고 내쉬며 내쉬며 내쉬며 내쉬며 내쉬며 내쉬
<br>
* 하지만 여전히 반복이;; 그리고 공감적 대화가 아님


---

**kogpt2**



```
# training_args = TrainingArguments(
#     output_dir="./kogpt2-therapy-chatbot",
#     overwrite_output_dir=True,
#     num_train_epochs=7,
#     per_device_train_batch_size=2,
#     save_steps=10_000,
#     save_total_limit=2,
#     logging_dir="./logs",
# )
```



loss가 47500	0.239600이지만 과적합<br>
결과 요딴식<br>
불안과 우울증 증상을 관리하는 첫 번째 단계 중 하나는 좋은 자기 관리 루틴을 확립하는 것입니다. 가장 기본적인 것들부터 시작하세요: 균형 잡힌 식사, 6-8시간의 수면, 그리고 하루에 최소 30분의 운동. 이것은 기분에 영향을 미치는 화학적 불균형을 조절하는 데 도움이 되며, 운동은 생각으로부터 벗어나 스트레스를 해소할 시간을 제공합니다. 기본적인 자가 관리를 시작하는 것이 어려운 것처럼 느껴질 수도 있지만, 이는 또한 감정의 강도를 줄이고, 실제로 정신 건강 문제를 야기하는 원인을 해결하는 데도 도움을 줍니다. 약물도 이러한 감정을 완화해줍니다.<br>
Context: 저는 극심한 불안을 가지고 있으며 모든 것을 시도해보지 않습니다. 남편만이 나를 돕고자 했지만 그는 신경 쓰지 않는다고 합니다.<br>

이 고통을 어떻게 이겨내고 혼자서 행복해지는 방법을 배울 수 있을까요?<br>
Resbook for Behavioraling Yourself at Work and in Relationships, Makiny (www.LivingYes, a Handweenry<br>

In [ ]:
# import pandas as pd
# from datasets import Dataset
# from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# # CSV 파일 로드
# df = pd.read_csv('/content/drive/MyDrive/[BITAmin24S]Chatbot/merged_df.csv')

# # 데이터 전처리 함수 정의
# def preprocess_data(row):
#     return f"Context: {row['context']}\nResponse: {row['response']}"

# df['text'] = df.apply(preprocess_data, axis=1)

# # Hugging Face Dataset으로 변환
# dataset = Dataset.from_pandas(df[['text']])

# # 토크나이저 로드
# tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>')

# 데이터셋 토크나이징
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# DataCollator 설정
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 모델 로드
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

# 파인튜닝 설정
training_args = TrainingArguments(
    output_dir="./kogpt2-therapy-chatbot",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
    evaluation_strategy="steps",
    eval_steps=1_000,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    weight_decay=0.01,
    learning_rate=5e-5,
    warmup_steps=1_000,
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# 모델 학습
trainer.train()


Map:   0%|          | 0/8058 [00:00<?, ? examples/s]

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

---

In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import pandas as pd
from datasets import Dataset
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# CSV 파일 로드
df = pd.read_csv('/content/drive/MyDrive/[BITAmin24S]Chatbot/merged_df.csv')

# 데이터 전처리 함수 정의
def preprocess_data(row):
    return f"Context: {row['context']}\nResponse: {row['response']}"

df['text'] = df.apply(preprocess_data, axis=1)

# Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df[['text']])

# 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>')

# eos_token을 pad_token으로 설정
tokenizer.pad_token = tokenizer.eos_token
# 또는 새로운 pad_token 추가
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# model.resize_token_embeddings(len(tokenizer))

# 데이터셋 토크나이징
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# DataCollator 설정
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 모델 로드

# # 파인튜닝 설정
# # training_args = TrainingArguments(
# #     output_dir="./kogpt2-therapy-chatbot",
# #     overwrite_output_dir=True,
# #     num_train_epochs=3,
# #     per_device_train_batch_size=4,
# #     save_steps=10_000,
# #     save_total_limit=2,
# #     logging_dir="./logs",
# #     logging_steps=500,
# #     evaluation_strategy="no",
# #     eval_steps=1_000,
# #     load_best_model_at_end=True,
# #     metric_for_best_model="eval_loss",
# #     greater_is_better=False,
# #     weight_decay=0.01,
# #     learning_rate=5e-5,
# #     warmup_steps=1_000,
# # # )

# # training_args = TrainingArguments(
# #     output_dir="./kogpt2-therapy-chatbot",
# #     overwrite_output_dir=True,
# #     num_train_epochs=3,
# #     per_device_train_batch_size=4,
# #     save_steps=10_000,
# #     save_total_limit=2,
# #     logging_dir="./logs",
# #     logging_steps=500,
# #     # Remove or set evaluation_strategy to "no"
# #     evaluation_strategy="no",
# #     load_best_model_at_end=True, # Add this line
# #     weight_decay=0.01,
# #     learning_rate=5e-5,
# #     warmup_steps=1_000,
# # )

# training_args = TrainingArguments(
#     output_dir="./kogpt2-therapy-chatbot",
#     overwrite_output_dir=True,
#     num_train_epochs=3,
#     per_device_train_batch_size=4,
#     save_steps=10_000,
#     save_total_limit=2,
#     logging_dir="./logs",
#     logging_steps=500,
#     # Disable evaluation
#     evaluation_strategy="no",
#     # Disable loading the best model
#     load_best_model_at_end=True,
#     weight_decay=0.01,
#     learning_rate=5e-5,
#     warmup_steps=1_000,
# )

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/[BITAmin24S]Chatbot/merged_df.csv'

In [ ]:
# Split the dataset into train and eval
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [ ]:
# Tokenize the datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

In [ ]:
# Basic
import torch
from transformers import GPT2LMHeadModel

# 모델 로드
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

# 평가 모드로 전환
model.eval()

# 데이터셋에서 하나의 배치를 가져오기 (예: 첫 번째 배치)
batch = tokenized_train_dataset['train'][:1]  # 첫 번째 데이터 포인트를 가져옵니다.
input_ids = torch.tensor(batch['input_ids'])
attention_mask = torch.tensor(batch['attention_mask'])

# 모델을 통해 손실 계산
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
    loss = outputs.loss
    logits = outputs.logits

# 모델 학습
trainer.train()

KeyError: "Column train not in the dataset. Current columns in the dataset: ['input_ids', 'token_type_ids', 'attention_mask']"

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./kogpt2-therapy-chatbot",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
    evaluation_strategy="no",  # Disable evaluation
    load_best_model_at_end=False,  # Disable loading the best model
    weight_decay=0.01,
    learning_rate=5e-5,
    warmup_steps=1_000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,  # Use the tokenized training dataset
    data_collator=data_collator,
    # Remove the callbacks if there's no evaluation
)

In [3]:
# 모델 학습
trainer.train()

NameError: name 'trainer' is not defined

In [ ]:
def chatbot_response(context):
    input_text = f"{context}\nResponse:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to('cuda') # Move input_ids to GPU

    # Generate response with a combination of strategies
    output = model.generate(
        input_ids,
        max_length=200,
        pad_token_id=tokenizer.eos_token_id,
        top_p=0.9,  # Nucleus sampling
        top_k=50,   # Top-k sampling
        temperature=0.7,  # Control randomness
        repetition_penalty=1.5,  # Penalize repeated phrases
        no_repeat_ngram_size=3   # Avoid repeating n-grams of size 3
    )

    # Decode and process the output
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract only the part after "Response:" and remove unwanted context
    response_start = generated_text.find("Response:") + len("Response:")
    response = generated_text[response_start:].strip()

    # Ensure the context is removed if present
    if response.startswith(context):
        response = response[len(context):].strip()

    return response

print(chatbot_response("요즘 너무 불안해요."))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


저도 그게 궁금해요. 괜찮을 거예요! 제가 하는 일이 정말 잘 될 수 있도록 도와드릴까요, 아니면 그냥 행복할 수가 있을까요?

어떻다면 제 자신을 발견하게 돼요(동물이 더 좋아질지도 모르겠습니다). 하지만 실제로 그렇게 하고 있는 것은 아니에요. 당신이 묘사한 많은 동물들은 자신이 어떻게 행동하는지에 대해 매우 신나 있어요. 그래서 다음 단계로 나아가고 싶어요.
1. 자기 자신에게 시간을 주어 그들이 왜 그런 행동을 취하는지 알아보세요. 2. 사랑하고 존중받는다고 느끼며 성장하도록 노력해보세요.
3. 자신에게도 시간이 필요하지만 지금 이 순간에도 여전히 충분하지 않다고 느껴지면 다시 집중하실 수도 있습니다. 휴식을 취하고, 무엇이 가장 힘들지, 얼마나 많이 피곤했는지 등을 기록해보고, 혼자 놀이나 운동 외의 다른 활동을 시도해볼 만한 여지를 만드십시오. 이렇게 하면 도움이 됩니다. 운동을 하면서 얻는 것이 있고, 그렇지 않더라도 단지 즐길 뿐이에요. 그러면 오히려 긍정적인 변화를 일으킬 가능성이 높아


In [ ]:

# Repository 생성 & model upload
REPO_NAME = YOUR_REPO_NAME # ex) 'my-bert-fine-tuned'
AUTH_TOKEN = YOUR_OWN_TOKEN # <https://huggingface.co/settings/token>

## Upload to Huggingface Hub
model.push_to_hub(
    REPO_NAME,
    use_temp_dir=True,
    use_auth_token=AUTH_TOKEN
)
tokenizer.push_to_hub(
    REPO_NAME,
    use_temp_dir=True,
    use_auth_token=AUTH_TOKEN
)
